In [14]:
import os
import pandas as pd
import json

def build_preference_json(strategy_name, excel_file, prompt_folder, gpt_folder, llama_folder, output_json):
    """
    Build JSON preference pairs for a single strategy with actual question text.
    GPT and Llama outputs correspond to prompt index - 1.
    """
    # Load Excel file
    df = pd.read_excel(excel_file)
    
    if 'prompt' not in df.columns:
        raise ValueError(f"'prompt' column not found in {excel_file}")

    preference_pairs = []

    for idx, row in df.iterrows():
        prompt_file = row['prompt']  # e.g., question01.txt
        # Extract digits and compute corresponding GPT/Llama file index
        prompt_num = int(''.join(filter(str.isdigit, prompt_file)))
        output_num = prompt_num - 1  # GPT/Llama use previous index

        # Paths
        prompt_path = os.path.join(prompt_folder, prompt_file)
        gpt_path = os.path.join(gpt_folder, f"output{output_num:02d}.txt")
        llama_path = os.path.join(llama_folder, f"output{output_num:02d}.txt")

        # Read files safely
        if not os.path.exists(prompt_path):
            print(f"[{strategy_name}] Missing prompt file: {prompt_path}, skipping...")
            continue
        if not os.path.exists(gpt_path):
            print(f"[{strategy_name}] Missing GPT file: {gpt_path}, skipping...")
            continue
        if not os.path.exists(llama_path):
            print(f"[{strategy_name}] Missing Llama file: {llama_path}, skipping...")
            continue

        # Read and clean text
        with open(prompt_path, 'r', encoding='utf-8') as f:
            question = f.read().replace('\n', ' ').strip()
        with open(gpt_path, 'r', encoding='utf-8') as f:
            chosen = f.read().replace('\n', ' ').strip()
        with open(llama_path, 'r', encoding='utf-8') as f:
            rejected = f.read().replace('\n', ' ').strip()

        if not question or not chosen or not rejected:
            print(f"[{strategy_name}] Empty content for prompt {prompt_file}, skipping...")
            continue

        # Construct JSON object
        pair = {
            "prompt": prompt_file,
            "question": question,
            "chosen": chosen,
            "rejected": rejected
        }
        preference_pairs.append(pair)

    # Save to JSON
    with open(output_json, 'w', encoding='utf-8') as f:
        json.dump(preference_pairs, f, indent=4, ensure_ascii=False)

    print(f"[{strategy_name}] JSON saved: {output_json} with {len(preference_pairs)} pairs.")


# Example usage
strategies = ["NoCoT", "CoT", "Divide&Conquer"]

for strategy in strategies:
    excel_file = f"Preference_Pair_{strategy}.xlsx"
    prompt_folder = os.path.join("prompt", strategy)
    gpt_folder = os.path.join("GPT_output", strategy)
    llama_folder = os.path.join("Llama3.1_output", strategy)
    output_json = f"DPO_{strategy}.json"

    build_preference_json(strategy, excel_file, prompt_folder, gpt_folder, llama_folder, output_json)


[NoCoT] JSON saved: DPO_NoCoT.json with 29 pairs.
[CoT] JSON saved: DPO_CoT.json with 28 pairs.
[Divide&Conquer] JSON saved: DPO_Divide&Conquer.json with 32 pairs.
